In [1]:
%run transformer_text_classifier.ipynb

In [20]:
SEQ_LENGTH = 128
FINETUNING_LEARNING_RATE = 1e-3

In [3]:
import os
import re
import math
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.utils import Sequence

In [12]:
batch_sentences = [
    'This is positive',
    'This is also positive',
    'a test and',
    'a test haha',
    'a test no',
    'a test oh no',
    'a test but',
    'a test cool',
]

label = np.array([
    [0,1],
    [0,1],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
])

In [13]:
batch = tokenizer(
    batch_sentences, 
    padding=True, 
    truncation=False, 
    return_tensors="tf")

input_ids = tf.keras.utils.pad_sequences(
    batch['input_ids'],
    maxlen=SEQ_LENGTH,
    dtype='int32',
    padding='post',
    truncating='post',
    value=0.0
    )

In [14]:
class TextSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

In [15]:
data_sequence = TextSequence(
    input_ids,
    label,
    batch_size = 2
    )

# training model 

In [21]:
model = transformer_text_single_label_classifier(
    num_label = 2,
    seq_length = SEQ_LENGTH,
    )

In [22]:
model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=FINETUNING_LEARNING_RATE),
    metrics=[    
        tf.keras.metrics.AUC(name = 'au-roc',),
        tf.keras.metrics.AUC(curve='PR', name = 'au-prc',),
        tf.keras.metrics.PrecisionAtRecall(recall = 0.9, name = 'precision'),    
    ],
)

In [23]:
model.fit(
    data_sequence,
    class_weight = {0:1,1:10},
    epochs = 50,
    batch_size = 8,
    shuffle= True,
    )

Epoch 1/50
4/4 [==============================] - 1s 49ms/step - loss: 14.0569 - au-roc: 0.1875 - au-prc: 0.3422 - precision: 0.5000
Epoch 2/50
4/4 [==============================] - 0s 53ms/step - loss: 3.8740 - au-roc: 0.1875 - au-prc: 0.3422 - precision: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 48ms/step - loss: 4.4104 - au-roc: 0.2500 - au-prc: 0.3651 - precision: 0.5000
Epoch 4/50
4/4 [==============================] - 0s 51ms/step - loss: 2.9136 - au-roc: 0.1875 - au-prc: 0.3422 - precision: 0.5000
Epoch 5/50
4/4 [==============================] - 0s 49ms/step - loss: 8.1623 - au-roc: 0.3750 - au-prc: 0.4126 - precision: 0.5000
Epoch 6/50
4/4 [==============================] - 0s 50ms/step - loss: 1.5526 - au-roc: 0.5625 - au-prc: 0.5199 - precision: 0.5000
Epoch 7/50
4/4 [==============================] - 0s 50ms/step - loss: 2.7956 - au-roc: 0.3125 - au-prc: 0.3891 - precision: 0.5000
Epoch 8/50
4/4 [==============================] - 0s 47ms/step - loss: 3.50

# end